In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

from natsort import natsorted
from dotenv import find_dotenv


root_dir = os.path.dirname(find_dotenv())

In [2]:
root_dir

'/home/bombra/library/padme'

In [3]:
from src import DMD
from src import snapshots_assembly
from src import (
    load_notebook_params,
    generate_paths,
    download_dataset,
    unpack_kaggle_dataset,
)

from src import PostProcessingDMD

In [4]:
notebook_params = load_notebook_params()

Before downloading dataset, you need to create an account on Kaggle. Instructions for using the Kaggle API can be seen at https://www.kaggle.com/docs/api
This dataset contains 991MB and two versions of the simulation covid spread lombardy etc

In [5]:
example_dataset = "seird"

dict_paths = generate_paths(example_dataset, root_dir, notebook_params)
dict_paths

download_dataset(dict_paths, example_dataset, notebook_params, FORCE_DOWNLOAD=False)
unpack_kaggle_dataset(dict_paths)

We need to unzip etc etc etc
points to data folder

Let's instantiate the dataset etc etc etc

In [6]:
os_walk_files = next(os.walk(dict_paths["snapshots_filepath"]))
folders = natsorted(os_walk_files[1])

filenames = [
    dict_paths["snapshots_filepath"]
    / Path(snapshot_folder)
    / Path(f"out_1_000_{str(i).zfill(5)}.h5")
    for i, snapshot_folder in enumerate(folders)
]

In [7]:
filenames[:5]

[PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step0/out_1_000_00000.h5'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step1/out_1_000_00001.h5'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step2/out_1_000_00002.h5'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step3/out_1_000_00003.h5'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step4/out_1_000_00004.h5')]

In [8]:
snapshot_ingestion_parameters = {
    "filenames": filenames,
    "dataset": "s",
}

In [9]:
dataset = snapshots_assembly(
    "h5_libmesh", snapshot_ingestion_parameters=snapshot_ingestion_parameters
)

2023-04-16 20:08:19,568 - src.preprocessing.build_snapshots - INFO - Starting choice of file type:
2023-04-16 20:08:19,569 - src.preprocessing.build_snapshots - INFO - libMesh/EdgeCFD HDF5 file selected.


In [10]:
dataset.shape

(13158, 482)

In [11]:
example_dataset = "sird"

dict_paths = generate_paths(example_dataset, root_dir, notebook_params)
dict_paths

download_dataset(dict_paths, example_dataset, notebook_params, FORCE_DOWNLOAD=False)
unpack_kaggle_dataset(dict_paths)

In [12]:
os_walk_files = next(os.walk(dict_paths["snapshots_filepath"]))
files = natsorted(os_walk_files[2])

filenames = [
    dict_paths["snapshots_filepath"] / Path(f"{str(i)}infected.vtk")
    for i, _ in zip(range(4, 480, 4), files)
]

In [13]:
filenames[:5]

[PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/4infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/8infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/12infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/16infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/20infected.vtk')]

In [14]:
filenames[-5:]

[PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/460infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/464infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/468infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/472infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/476infected.vtk')]

In [15]:
snapshot_ingestion_parameters = {
    "filenames": filenames,
    "starting_line": 125939,
    "ending_line": 210239,
}

In [16]:
# dataset = snapshots_assembly(
#     "vtk_freefem", snapshot_ingestion_parameters=snapshot_ingestion_parameters
# )

2023-04-16 20:08:20,286 - src.preprocessing.build_snapshots - INFO - Starting choice of file type:
2023-04-16 20:08:20,287 - src.preprocessing.build_snapshots - INFO - FreeFem++ vtk file selected.


In [ ]:
dmd_parameters = {
    "factorization_algorithm": "randomized_svd",
    "basis_vectors": 50,
    "randomized_svd_parameters": {
        "power_iterations": 1,
        "oversampling": 20,
    },
    "starting_step": 20,
    "dt_simulation": 0.05,
}

In [ ]:
dmd = DMD(dataset, dmd_parameters)

In [ ]:
dmd.factorization()

In [ ]:
dmd.dmd_approximation["s"]

In [ ]:
dmd.dmd_core()

In [ ]:
import numpy as np


def compute_frobenius_norm(mat_a, mat_b):
    return np.linalg.norm(mat_a - mat_b) / np.linalg.norm(mat_a)

In [ ]:
compute_frobenius_norm(dmd.snapshots_matrix, dmd.dmd_approximation["dmd_matrix"])

In [ ]:
dmd.dmd_approximation.keys()

In [ ]:
dmd_visualizer = PostProcessingDMD(dmd.dmd_approximation)

In [ ]:
dmd.dmd_approximation["s"].shape

In [ ]:
dmd_visualizer.plot_singular_values("plotly")

In [ ]:
dmd.dmd_approximation["eigenvals_original"]

In [ ]:
dmd_visualizer.plot_eigenvalues("plotly")

In [ ]:
dmd.dmd_approximation.keys()